# TAR: Taller de Aprendizaje por Refuerzo 2025
## Laboratorio 5: Gymnasium + stable_baselines3

### Stable Baselines3 (SB3) es una libreria algoritmos de reinforcement learning implementada en PyTorch. 

*Ejecutar esta celda solo la primera vez (si estan usando un entorno local - es la misma del lab 4) para descargar e instalar los paquetes necesarios. Si ejecutan el notebook en colab tendran que ejecutarla cada vez que reinicien el kernel*

In [ ]:
# !apt install swig cmake
# !pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [1]:
import subprocess

print(subprocess.run(["cmake", "--version"], capture_output=True).stdout.decode())
print(subprocess.run(["swig", "-version"], capture_output=True).stdout.decode())

cmake version 4.0.2

CMake suite maintained and supported by Kitware (kitware.com/cmake).


SWIG Version 4.3.1

Compiled with x86_64-w64-mingw32-g++ [x86_64-w64-mingw32]

Configured options: +pcre

Please see https://www.swig.org for reporting bugs and further information



In [2]:
!pip install Box2D 

In [3]:
import Box2D
print(Box2D.__version__)

2.3.10


In [4]:
!pip install "torch>=1.11" "gymnasium==0.28.1" huggingface-hub~=0.8 wasabi 

In [5]:
!pip install gymnasium 

In [6]:
!pip install pygame --only-binary :all: 

In [7]:
!pip install stable-baselines3==2.0.0a5 huggingface_sb3 swig 

In [8]:
!pip install ipywidgets 

In [9]:
import importlib
import sys

# Paquetes a verificar con sus nombres de importación
required_packages = {
    "gymnasium": "gymnasium",
    "Box2D": "Box2D",
    "pygame": "pygame",
    "stable_baselines3": "stable_baselines3",
    "huggingface_sb3": "huggingface_sb3",
    "swig": "swig"
}

print("🔎 Verificando dependencias:\n")

for name, module in required_packages.items():
    try:
        mod = importlib.import_module(module)
        version = getattr(mod, "__version__", "✅ (versión no disponible)")
        print(f"{name:20} ➜ ✅ Encontrado, versión: {version}")
    except ModuleNotFoundError:
        print(f"{name:20} ➜ ❌ No encontrado")
    except Exception as e:
        print(f"{name:20} ➜ ⚠️ Error: {e}")

print("\n✅ Verificación completa.")

🔎 Verificando dependencias:

gymnasium            ➜ ✅ Encontrado, versión: 0.28.1
Box2D                ➜ ✅ Encontrado, versión: 2.3.10
pygame               ➜ ✅ Encontrado, versión: 2.6.1
stable_baselines3    ➜ ✅ Encontrado, versión: 2.0.0a5
huggingface_sb3      ➜ ✅ Encontrado, versión: ✅ (versión no disponible)
swig                 ➜ ✅ Encontrado, versión: 4.3.1

✅ Verificación completa.


In [10]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="rgb_array")
obs, info = env.reset()
print("Ambiente cargado 🎯")

Ambiente cargado 🎯


In [15]:
!python --version

Python 3.11.11


#### IMPORTS

In [16]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Ejercicio 1. Lunar Landing

En este ejemplo entrenaremos un agente que aprenda a aterrizar correctamente una nave en la luna.
Doc del ambiente: https://gymnasium.farama.org/environments/box2d/lunar_lander/


#### 1.1 Estudiar el ambiente

In [27]:
#TODO Definir el ambiente `LunarLander-v2` e imprimir el tamaño del espacio de observación y del espacio de acciones.

env = gym.make("LunarLander-v2", continuous=False, gravity=-1.62,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5)
env.reset()

tamaño_obs = env.observation_space.shape
print("Tamaño del espacio de observación: ", tamaño_obs)

tamaño_acciones = env.action_space.n
print("Tamaño del espacio de acción: ", tamaño_acciones)

Tamaño del espacio de observación:  (8,)
Tamaño del espacio de acción:  4


El vector de **observación** corresponde a los siguientes parámetros de la nave:
- posición en x
- posición en y
- velocidad lineal en x
- velocidad lineal en y
- ángulo
- velocidad angular
- variable binaria que representa si la pata izquierda toca el suelo o no
- variable binaria que representa si la pata derecha toca el suelo o no


Por otro lado, el espacio de **acciones** es:

- Acción 0: no hacer nada
- Acción 1: Encender el motor lateral izquierdo (empuja a la derecha)
- Acción 2: Encender el motor principal (empuja hacia arriba)
- Acción 3: Encender el motor lateral derecho (empuja a la izquierda)


Función de **recompensa**:

  Después de cada paso, se otorga una recompensa. La recompensa total de un episodio es la suma de las recompensas de todos los pasos dentro de ese episodio.

  Para cada paso, la recompensa:

- se incrementa/decrementa cuanto más cerca/más lejos esté la nave de la zona de aterrizaje
- se incrementa/decrementa cuanto más lento/más rápido se mueva la nave
- se decrementa cuanto más se incline la nave hacia los costados
- se incrementa por 10 puntos para cada pata que toque el piso
- se decrementa por 0.03 puntos por cada frame en que se use motor lateral
- se decrementa por 0.3 puntos por cada frame en que se use el motor principal

El episodio recibe una recompensa adicional de -100 si choca y +100 si aterriza de manera segura.

Un episodio termina si:
- la nave choca (el cuerpo principal, no las patas, de la nave entra en contacto con la luna)
- la nave se sale del área visible (x sale de su rango)
- la nave está en "modo dormido" (un cuerpo en "modo dormido" es uno que no se mueve y no choca con ningún otro cuerpo)

##### Multi-procesamiento
A continuación se vectoriza el ambiente para poder paralelizar el entrenamiento, (por suerte la libreria nos da las heramientas para lograr esto sin mucho esfuerzo)

In [29]:
from stable_baselines3.common.env_util import make_vec_env

# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

#### 1.2 Definir el modelo usando el modelo [Proximal Policy Optimization(PPO) de stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)

Leer la documentación y variar diferentes hiperparámetros como `batch_size`, `gamma`, entre otros.

¿Que tipo de metodo implementa?

##### RESPUESTA --> 
PPO es un algoritmo de optimización de políticas:

* Es on-policy (aprende de las acciones que realmente toma).
* Pertenece a la familia de métodos de policy gradient.
* Es una mejora de TRPO (Trust Region Policy Optimization), pero más simple.
* Usa técnicas como clipping para limitar cuánto puede cambiar la política en cada actualización y así estabilizar el entrenamiento.

In [ ]:
from stable_baselines3 import PPO

# model = PPO("MlpPolicy", env, verbose=1)

model = PPO(
    policy="MlpPolicy", # MLP Perceptron
    env=env,
    verbose=1,
    batch_size=64,
    gamma=0.99,
    n_steps=2048,
    ent_coef=0.01,
    learning_rate=3e-4
)

In [ ]:
!pip install comet-ml

In [ ]:
from comet_ml import Experiment

experiment = Experiment(
    api_key="TU_API_KEY",
    project_name="lunar-lander",
    workspace="TU_USUARIO"
)

In [ ]:
# Inicialización del experimento (reemplazá con tu clave y proyecto)
experiment = Experiment(
    api_key="olHeoTp6comM3x5X987X7RuMm",
    project_name="taller8",
    workspace="florencia-roque"
)

# Registrar métricas
experiment.log_metric(f"{dataset_name}_naive_MAE", mae)
experiment.log_metric(f"{dataset_name}_naive_RMSLE", rmsle)

experiment.set_name("Naive Forecasting")
experiment.add_tag("baseline")
experiment.add_tag("naive")
experiment.log_other("descripcion", "Modelo base naive forecasting usando y[t] ≈ y[t−1]

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class CometCallback(BaseCallback):
    def __init__(self, experiment, verbose=0):
        super().__init__(verbose)
        self.experiment = experiment

    def _on_step(self):
        self.experiment.log_metric("reward", self.locals["rewards"][0], step=self.num_timesteps)
        return True


In [ ]:
model.learn(total_timesteps=100_000, callback=CometCallback(experiment))


In [ ]:
# Entrenar por 1.000.000 timesteps

#model.learn(total_timesteps=1000000)

model_name = "ppo-LunarLander-v2"

# Guardar el modelo
#model.save(model_name)


#### 1.3 Evaluación del modelo:

¿Como podemos evaluar si el agente aprendió a aterrizar la nave?

##### RESPUESTA --> 

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(gym.make("LunarLander-v2", render_mode='rgb_array'))
#mean_reward, std_reward = 
#print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

#### 1.4 Correr el agente entrenado y guardar un video del aterrizaje

si esto da error de timeout pueden generar un .py y correrlo desde consola para guardar el video

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import RecordVideo

# Load the trained model
model = PPO.load("ppo-LunarLander-v2")

# Create the environment with the proper render_mode for video recording
env = DummyVecEnv([lambda: RecordVideo(gym.make("LunarLander-v2", render_mode="rgb_array"), "./videos/PPO", episode_trigger=lambda x: True)])

# Reset the environment to get the initial observation
obs = env.reset()
done = False

# Play one episode to capture the video
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
# Close the environment to save the video file
env.close()

print("Video saved in the './videos' directory.")


## Ejercicio 2. Cart Pole

En este ejemplo entrenaremos un agente que aprenda a equilibrar un poste sobre un carrito que se mueve sobre una linea.
Doc del ambiente: https://gymnasium.farama.org/environments/classic_control/cart_pole/


#### 2.1 Estudiar el ambiente:

¿Como son el espacio de acciones y el espacio de estados?

##### RESPUESTA -->


In [ ]:
#TODO Definir el ambiente e imprimir el tamaño del espacio de observación y del espacio de acciones.


#### 2.2 Definir el modelo usando el modelo - A2C de stable_baselines3

¿Que tipo de metodo implementa?

##### RESPUESTA --> 

In [ ]:
#TODO definir el modelo A2C 

#### 2.3 Vectorizar el ambiente y entrenar el modelo

In [ ]:
#TODO vectorizar el ambiente y entrenar el agente A2C

#...

model_name = "A2C-CartPole-v1"

# Guardar el modelo
#model.save(model_name)

##### ¿Comó se si mi modelo aprendió?

##### ¿Comó son las recompensas?

### RESPUESTA:

#### 2.4 Evaluar el agente


In [ ]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import RecordVideo

# Load the trained model
model = A2C.load("A2C-CartPole-v1")

# Create the environment with the proper render_mode for video recording
env = DummyVecEnv([lambda: RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"), "./videos/A2C", episode_trigger=lambda x: True)])

# Reset the environment to get the initial observation
obs = env.reset()
done = False

# Play one episode to capture the video
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
# Close the environment to save the video file
env.close()

print("Video saved in the './videos' directory.")